In [1]:
!pip install scikit-learn=='1.2.1'

In [21]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib
from sagemaker.sklearn import SKLearnModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

In [4]:
df = pd.read_csv("data/original/heart.csv")

In [10]:
X = df.drop("output", axis=1)

In [11]:
y = df["output"]

In [40]:
model = RandomForestClassifier(max_depth=5)

In [41]:
model.fit(X,y)

RandomForestClassifier(max_depth=5)

In [42]:
joblib.dump(model, "model.joblib")

['model.joblib']

In [44]:
!tar -czvf model.tar.gz model.joblib

model.joblib


In [45]:
!aws s3 cp model.tar.gz s3://heart-custom-model-gfbg/heart_custom_model/

upload: ./model.tar.gz to s3://heart-custom-model-gfbg/heart_custom_model/model.tar.gz


In [46]:
model = SKLearnModel (
    model_data = "s3://heart-custom-model-gfbg/heart_custom_model/model.tar.gz",
    role="LabRole",
    entry_point="entrypoint.py",
    framework_version="1.2-1"
)

In [47]:
model.deploy(initial_instance_count=1, 
             instance_type="ml.m5.xlarge",
             serializer=CSVSerializer(),
             deserializer=CSVDeserializer())

------!